In [1]:
from Core import Workspace
import Logger

logger = Logger.console_logger(0, 'DEBUG')
context = Workspace('/Users/ataymano/data/ws/5', logger)

In [5]:
def generate_something(value1, value2):
    return {'key1': value1, "key2": value2}

In [9]:
context.run(generate_something, 5, 12)

[0][13-05-2020 10:37:13]Generating path: (generate_something,5,12)	769c133f969e460ed646cf19efcbf8c51dd6893fa5ff01b301e795b87d4ce8db
[0][13-05-2020 10:37:13]Result of generate_something is found


{'key1': 5, 'key2': 12}

In [10]:
vw_path = '/Users/ataymano/src/github/VowpalWabbit/vowpal_wabbit/build/vowpalwabbit/vw'
from Vw import Vw, VwInput
vw = Vw(vw_path, context)

In [12]:
opts = {'#cmd': '--ccb_explore_adf --epsilon 0.1 --dsjson', '-d': '/Users/ataymano/data/1.json'}
vw.run(opts)

[0][13-05-2020 10:39:05]Generating path: (__run__,/Users/ataymano/src/github/VowpalWabbit/vowpal_wabbit/build/vowpalwabbit/vw --ccb_explore_adf --epsilon 0.1 --dsjson -d /Users/ataymano/data/1.json)	c9151f6b065c7a76f38c75d09bd01c661c4fc2482a67ef22fdc61a7feacebdc5
[0][13-05-2020 10:39:05]Result of __run__ is found


{'average loss': -0.011934}

In [ ]:
inputs = ['/Users/ataymano/data/1.json', '/Users/ataymano/data/2.json']

opts_in = {'#base': '--ccb_explore_adf --epsilon 0.3 --dsjson'}
opts_out = ['--cache_file', '-f']

result = vw.test(inputs, opts_in, opts_out)
result.Loss

In [ ]:
result.Populated

In [ ]:
inputs = ['/Users/ataymano/data/1.json', '/Users/ataymano/data/2.json']

opts_in = {'#base': '--ccb_explore_adf --epsilon 0.3 --dsjson --save_resume'}
opts_out = ['-f']

result = vw.train(inputs, opts_in, opts_out)
result.Loss

In [ ]:
from Pool import MultiThreadPool
opts_in_1 = {'#base': '--ccb_explore_adf --epsilon 0.3 --dsjson --save_resume'}
opts_in_2 = {'#base': '--ccb_explore_adf --epsilon 0.31 --dsjson --save_resume'}
opts_out = ['-f']
pool = MultiThreadPool(2)
results = pool.map(vw.train, [(inputs, opts_in_1, opts_out), (inputs, opts_in_2, opts_out)])

In [ ]:
from VwSweep import VwSweep
import VwOptsGrid

sweep = VwSweep(vw)

In [ ]:
opts = [{'#base': '--ccb_explore_adf --epsilon 0.3 --dsjson --save_resume'},
    {'#base': '--ccb_explore_adf --epsilon 0.31 --dsjson --save_resume'}]
results = sweep.iteration(opts, inputs)

In [ ]:
results[0].Opts

In [ ]:
from VwOptsGrid import dimension, product, Grid, Configuration
def generate_grid():
    hyper_points_1 = product(
        dimension('-l', [1e-6, 1e-5]),
        dimension('--cb_type', ['ips', 'mtr']),
    )
    hyper_points_2 = dimension('--l1', [1e-09, 1e-07])

    return [
        Grid(hyper_points_1, Configuration(name='Step1', output=1, promote=1)),
        Grid(hyper_points_2, Configuration(name='Step2', output=1, promote=1))
    ]

In [ ]:
grid = generate_grid()
base_command = {'#base': '--ccb_explore_adf --epsilon 0.2 --dsjson --save_resume --preserve_performance_counters'}

result = sweep.run(grid, inputs, base_command)